# Marital Status and Number of Children Impact on Loan Default

## Introducation

The objective of this project is to prepare a report for the bank's loan division, which determines whether the customer's marital status and number of children have an impact on whether they will default on a loan. The insights of the report will be considered later on when building a credit score for a potential customer.

## Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
credit_scoring = pd.read_csv('credit_scoring_eng.csv')
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Conclusion

1. There are 21521 rows, and 12 columns.
2. The days_employed column and the total_income column has  missing values. Also, we see that they are both quantitative.
3. We don't have null values.

## Data preprocessing

### Processing missing values

**First, let't see the first ten rows of the data**

In [3]:
credit_scoring.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


We see that the education column contains case-sensitive values. Let's give it a quick treatment. 😊

In [4]:
credit_scoring['education_clean'] = credit_scoring['education'].str.lower()
credit_scoring['education_clean'].head(10)

0      bachelor's degree
1    secondary education
2    secondary education
3    secondary education
4    secondary education
5      bachelor's degree
6      bachelor's degree
7    secondary education
8      bachelor's degree
9    secondary education
Name: education_clean, dtype: object

**Than let's check the number of missing values in each column**

In [5]:
credit_scoring.isnull().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
education_clean        0
dtype: int64

**Now, we will check the percentage of missing values** 

In [6]:
data_size = len(credit_scoring)
missing_per_column = credit_scoring[['days_employed', 'total_income']].isnull().sum()
missing_per_column / data_size

days_employed    0.100999
total_income     0.100999
dtype: float64

The percentage of the missing values in each column is around 10%. Although it is common to consider deleting the rows that contain missing data, if it's around or less than 10%, we will not do so. This is because those rows have information about other areas of interest.

**Let's check if there is a pattern or a connection between the 2 columns that can explain the cause for having the same amount of missing values.**

To do this, we will find the rows with the missing values in each column and examine them.

*days_employed column*
***

In [7]:
credit_scoring[credit_scoring['days_employed'].isnull()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_clean
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,secondary education
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,secondary education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,secondary education
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,secondary education
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,secondary education
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate,secondary education
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family,bachelor's degree
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate,bachelor's degree
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing,bachelor's degree
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing,secondary education


We see above that many of the missing values belong to customers with secondary degrees. Let's investigate it further.

In [8]:
credit_scoring[credit_scoring['days_employed'].isnull()]['education_clean'].value_counts()

secondary education    1540
bachelor's degree       544
some college             69
primary education        21
Name: education_clean, dtype: int64

Indeed we see that most of the missing values belong to customers with a secondary degree. Now let's go check the 'total_income' to see if there is the same pattern. 

*total_income columne*
***

In [9]:
credit_scoring[credit_scoring['total_income'].isnull()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_clean
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,secondary education
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,secondary education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,secondary education
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,secondary education
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,secondary education
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate,secondary education
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family,bachelor's degree
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate,bachelor's degree
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing,bachelor's degree
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing,secondary education


As is it was in 'days_emloyed', we see that many of the missing values in total_income column belong to customers with a secondary degree. Let's go check the numbers.

In [10]:
credit_scoring[credit_scoring['total_income'].isnull()]['education_clean'].value_counts()

secondary education    1540
bachelor's degree       544
some college             69
primary education        21
Name: education_clean, dtype: int64

The numbers are exactly the same in the 'days_employed' column. This means that there was probably a technical fault with the customers that have a secondary degree.

**Cleaning the days_employed and the total_income columns**


When we examined above the first 10 rows of credit_scoring DataFrame we saw that the 'days_employed' column has negative values. This implies that we need to check the values of these column.

*days_employed column*
***

The values in this column are in days. Let's first transform them to years for convenience and look at the results. 

In [11]:
year = 365
credit_scoring['days_employed'] = credit_scoring['days_employed'] / year
credit_scoring['days_employed'].head(10)

0    -23.116912
1    -11.026860
2    -15.406637
3    -11.300677
4    932.235814
5     -2.537495
6     -7.888225
7     -0.418574
8    -18.985932
9     -5.996593
Name: days_employed, dtype: float64

Here we see that the data in the 'days employed' are problematic.There are negative values (as we have seen before) and also big and unreasonable values (like 942 years) that make no sense. The 'days_employed' parameter does not influence our analysis, and we don't need it for it, so the best option in such case will be to leave the column as it is.

*total_income column*
***

Let's check the values in this column. Although the 10 first rows seem to be o.k, let's check the whole column just to be sure.
We know that this column is quantitative and a float type, so let's check if there are any negative values. For that purpose, we will use the min() method.

In [12]:
credit_scoring['total_income'].min()

3306.762

Here we see that the smallest value is 3306.762, and it's positive - Yey! 😁
    That means that the values that aren't missing are good to use for our analysis, and that is excellent news because the total_income column is one of our main variables of interest. 🥳  Now we "just" need to fill in the missing values.

**Before filling in the missing values in total_income, we need to see if other variables can help us understand how we should fill it.**

It's known that the level of income is related to the line of business. If you are unemployed or a student, logically your income will be smaller,  but if you are an entrepreneur or businesswoman, most chances are that your income will be high.
Let's use income_type to see if it's applied in our dataset. 

As we saw earlier, the total_income is a quantitative variable so we will group the data by income_type and find the mean and the median of each income_type, sorted in descending order for ours convenience.

In [13]:
print('These are the means of each type of income:')
credit_scoring.groupby('income_type')['total_income'].mean().sort_values(ascending = False)

These are the means of each type of income:


income_type
entrepreneur                   79866.103000
business                       32386.793835
civil servant                  27343.729582
employee                       25820.841683
retiree                        21940.394503
unemployed                     21014.360500
student                        15712.260000
paternity / maternity leave     8612.661000
Name: total_income, dtype: float64

In [14]:
print('These are the medians of each type of income:')
credit_scoring.groupby('income_type')['total_income'].median().sort_values(ascending = False)

These are the medians of each type of income:


income_type
entrepreneur                   79866.1030
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
unemployed                     21014.3605
retiree                        18962.3180
student                        15712.2600
paternity / maternity leave     8612.6610
Name: total_income, dtype: float64

We see from the outputs that students, retirees, and parents who are in paternity/maternity leaves earn the least, while entrepreneurs earn the most. Clearly, the type of income impacts the monthly income. Lucky we checked it out.  😎 

Also, we see that the mean of business, civil servant, employee, and retiree are higher than its median, while the other income types are equal. That means that these income types have outliers.  

Now let's check the missing values in total_income by income_type. We will use the Value_counts method to receive a descending list with the number of missing values for each type of income. 

In [15]:
print('Count of missing values in every type of income:')
credit_scoring.loc[credit_scoring['total_income'].isnull(), 'income_type'].value_counts()

Count of missing values in every type of income:


employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

The output above matches the results of the median and the mean we found earlier. The employee, business, retiree and civil servant income types had high outliers, so we will use the median to fill in the missing values. 

**Filling in the missing values**

First, let's find the rows with the missing values to check afterward if the filling was done correctly.

In [16]:
credit_scoring[credit_scoring['total_income'].isnull()].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_clean
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,secondary education
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,secondary education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,secondary education
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,secondary education
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,secondary education


In [17]:
credit_scoring[credit_scoring['total_income'].isnull()].tail(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_clean
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car,secondary education
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony,secondary education
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property,bachelor's degree
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate,secondary education
21510,2,NaN,28,secondary education,1,married,0,F,employee,0,NaN,car purchase,secondary education


Here we see that rows number 12, 26, and 21495 have missing values. We will remember that for later. 

Now will use the tranform method to fill in the missing values of the total_income column and create a new column for that with the name clean_income.  



In [18]:
credit_scoring['clean_income'] = credit_scoring['total_income'].fillna(credit_scoring.groupby('income_type')['total_income'].transform('median'))

Now we will check if the filling process was done correctly by examining the rows with the missing values that we found earlier.

In [19]:
credit_scoring.loc[(12,26, 21495), ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_clean,clean_income
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,secondary education,18962.3180
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,secondary education,24071.6695
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony,secondary education,22815.1035


We see here that filling the missing numbers by the median of income type was done successfully.

Now let's check and make sure that all the missing values have been filled:

In [20]:
credit_scoring.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
education_clean        0
clean_income           0
dtype: int64

No missing values in the clean_income column! 🥳 

### Conclusion

* The missing values that we detected in the total_income and days_employes columns were float type, meaning both of the variables are quantitative. So the filling of the missing values was done be with representative value -  median.
* A possible reason for the presence of missing values is that while transferring the data, some of them were omitted due to some technical glitch. The data shows that there probably was a technical fault with the customers that have a secondary degree.
* When checking the values in the days_employed column and transferring them to years, we saw that the data is problematic. It has negative numbers and unreasonable numbers (like 900+ years of work). Due to the reason we do not need this data for our analysis because it's not one of the variables of main interest, I decided to leave the values in the column as is and not touch it. 
* Filling the missing values in total_income column was made with the following steps:
1. Checking the data for negative values - all the values are positive 🎉 
2. Checking if the income type influences the total income in our data as it is in the real world - it is! The total income for every income type is different.
3. Finding the mean and the median of the total_income column by income_type column. The findings showed that the mean is higher than the median in 5 of 8 income_type groups, which tells us that there are high outliers that pulling the mean up. Therefore, the median is the right way to fill in the missing values. 
4. Filling the missing values of the total_income column in a new 'clean_income' column with median by a group of income_type with the transfer method. 


### Data type replacement

We saw in the first step of this analysis the general information of the data, which contains the types of every column. The type of the following columns is incorrect:
1. **children** - it's need to be of an integer type (nobody has half a kid 😅), but in the DataFrame it's float type. 
2. **dob_years** - Customer age. It may sometimes be important to present the exact age in months and days, but in this case, the age in years will suffice. So this variable also need to be integer.
3. **education_id** - contains whole numbers that represent different education levels. Also need to be an integer.
4. **family_status_id** - contains whole numbers that represent different family status. Also need to be an integer.
5. **debt** - contains whole numbers that tell us whether the customer has ever defaulted on a loan. 

**Let's turn them to integers! 🤩**

In [21]:
credit_scoring['children'] = credit_scoring['children'].astype(int)
credit_scoring['dob_years'] = credit_scoring['dob_years'].astype(int)
credit_scoring['education_id'] = credit_scoring['education_id'].astype(int)
credit_scoring['family_status_id'] = credit_scoring['family_status_id'].astype(int)
credit_scoring['debt'] = credit_scoring['debt'].astype(int)
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
 12  education_clean   21525 non-null  object 
 13  clean_income      21525 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB


### Conclusion

As we can see from the output above, we replaced the original data types to integer type in the relevant columns.Now it's reflects in the best way the reality, and we will be no problem making arithmetic actions with the values for our analysis. 

### Processing duplicates

The data in the file does not contain a unique variable that differentiates one client (or row) from another. So we can not really know if 2 identical rows are of the same person (i.e. duplicate data), or of two different people who happen to have the same data.That is, it is impossible to process duplicates on this data.

### Conclusion

Because it's impossible to handle duplicated in this data file, there might be a chance that our analysis will be biased and inaccurate. In real life, we will probably contact the bank and talk with the developers to check it out and send a new file with customers_id column.  

### Categorizing Data

In this section we will categorize the following columns:
 1. clean_income (a copy of total_income, but with fiiled values)
 2. purpose
 
We will also check the other relevant variables to see if there is a need to categorize them. 

*clean_income*
***

Let's divide the clean_income column into its quarterly distribution. To do so we will use the describe method, build a function and use the apply metohd to our data.

In [22]:
credit_scoring['clean_income'].describe()

count     21525.000000
mean      26436.051922
std       15686.986477
min        3306.762000
25%       17247.708000
50%       22815.103500
75%       31287.991000
max      362496.645000
Name: clean_income, dtype: float64

In [23]:
def income_category (x):
    if  0 <= x <= 17247.708000:
        return 'poor'
    elif 17247.708000 < x <= 22815.103500:
        return 'middle class'
    elif 22815.103500 < x <=  31287.991000:
        return 'rich'
    elif 31287.991000 < x:
        return 'millionaire'
    else:
        return 'other'

credit_scoring['grouped_income'] = credit_scoring['clean_income'].apply(income_category)    
credit_scoring['grouped_income'].value_counts()


middle class    5523
poor            5382
millionaire     5381
rich            5239
Name: grouped_income, dtype: int64

We see that the categories size are roughly the same and that the function worked. That's good! 🥳 

In [24]:
credit_scoring.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_clean,clean_income,grouped_income
0,1,-23.116912,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,bachelor's degree,40620.102,millionaire
1,1,-11.026860,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,secondary education,17932.802,middle class
2,0,-15.406637,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,secondary education,23341.752,rich
3,3,-11.300677,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,secondary education,42820.568,millionaire
4,0,932.235814,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,secondary education,25378.572,rich
5,0,-2.537495,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,bachelor's degree,40922.170,millionaire
6,0,-7.888225,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,bachelor's degree,38484.156,millionaire
7,0,-0.418574,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education,secondary education,21731.829,middle class
8,2,-18.985932,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding,bachelor's degree,15337.093,poor
9,0,-5.996593,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,secondary education,23108.150,rich


Looking great! 🤩 

*purpose column*
***

First, let's check the unique values of the purpose column. we will use the value_counts method to see the values and their quantity.

In [25]:
credit_scoring['purpose'].value_counts()

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

At a glance it is easy to see that there are 4 categories:
1. wedding
2. housing
3. car
4. education

Because there is a small number of values, we will build the list of the 4 categories manually.


In [26]:
wedding = ['wedding', 'ceremony']
housing = ['real', 'estate', 'property', 'renting', 'housing', 'house', 'residential' ]
car = ['car', 'cars', 'second-hand']
education = ['university', 'education', 'educated', 'higher education' ]

Now we will use stemmy lemmatization technique, build a function and use the apply method to our data.

In [27]:
m = Mystem()

#Building the lemmatization function
def lemmatization_func(x):
    lemmatized = m.lemmatize(x)
    lemmatized = [word.lower() for word in lemmatized]
    if any(word in lemmatized for word in wedding):
        return 'wedding'
    elif any(word in lemmatized for word in housing):
        return 'housing'
    elif any(word in lemmatized for word in car):
        return 'car'
    elif any(word in lemmatized for word in education):
        return 'education'
    else:
        return 'other'
    

#Using the apply method:
credit_scoring['grouped_purpose'] = credit_scoring['purpose'].apply(lemmatization_func)
credit_scoring['grouped_purpose'].value_counts()

housing      10840
car           4315
education     4022
wedding       2348
Name: grouped_purpose, dtype: int64

There's no other category - yay!

In [28]:
credit_scoring.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_clean,clean_income,grouped_income,grouped_purpose
0,1,-23.116912,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,bachelor's degree,40620.102,millionaire,housing
1,1,-11.026860,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,secondary education,17932.802,middle class,car
2,0,-15.406637,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,secondary education,23341.752,rich,housing
3,3,-11.300677,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,secondary education,42820.568,millionaire,education
4,0,932.235814,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,secondary education,25378.572,rich,wedding


Such beautiful categories!  😍 

We categorized the 2 columns that for sure needed to be divided by groups. Now let's check quickly the other variables that are relevant for our analysis:
* children
* family_status


*children column*
***

In [29]:
credit_scoring['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

The output shows that there are some strange values: -1 and 20. 
Logic says that maybe the -1 is actually 1, and there's a typing error, and maybe 20 is actually 2, also due to type error. 
Let's change those values:
* -1 ==> 1
* 20 ==> 2

Building a function that replaces the values and using the apply method on our data.

In [30]:
# Building the function
def children_num(x):
    if x == -1:
        return 1
    elif x == 20:
        return 2
    else:
        return x

# Use the apply method
credit_scoring['clean_children'] = credit_scoring['children'].apply(children_num)
credit_scoring['clean_children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: clean_children, dtype: int64

There are 6 groups in the children column, which is a pretty small amount so I don't think there is a need to categorize it.
But it's a good thing that we checked the values and fixed them!


*Family_status*
***

In [31]:
credit_scoring['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

The values seem excellent, and no trace of strange values. There are 5 categories, which is an acceptable amount of categories. We could to categorized it into people who are in a relationship and not in a relationship, but I am interested to know the difference between each status, so I leave it as is. 

### Conclusion

1. We categorized the income level of the customers to 4 groups:
* poor
* middle class
* rich
* millionaire

We did the classification according to the 4 percent of the column (25%, 50%, 75%). We used the description method to find the value for each quarterly percentage, built a function that assigns the value to the correct category, and applied it (using the apply method😁 ) to our data.

2. We also categorized the different purposes of the customers to 4 categories:
* wedding
* housing
* car
* education

We used the lemmatization technique:
* First we manually created the 4 categories due to the small number of categories. 
* Then we built a lemmatization function that assigns each given value to the right category. 
* In the final step we applied the lemmatization function that we built to our data.

It seems that the most common reason for taking a loan is housing purposes. Also, buying a car and acquiring education are issues of equal importance for customers. A wedding is the least uncommon cause of taking a loan.

Interesting priorities, people are willing to spend more money on a car than on their wedding, and the same amount of money for car and education. It seems that car is a very important feature in the customers of this bank. I suspect this bank is in a "low love life" suburban and non-urban area where customers rely on private vehicles rather than public transportation (Bummer 😕). 

3. We checked the children variable and saw that there are 2 odd values: -1 and 20. We replaced them with 1 and 2 respectively. The changes showed that most of the customers don't have children, and among those who do, most of them have only 1 child. (Kids are fun, start making them!😂)

4. We examined the family_status column and saw that the values are perfect with only 5 categories, so it is logically to leave them as it is and not categorizing. 

It seems that most of the customers are married, so weddings are good for the bank. Maybe the bank should promote better terms for wedding loans (this will repay itself later 😁). 

## Answering the following questions

- Is there a relation between having kids and repaying a loan on time?

We will use pivot_table method.

In [32]:
credit_scoring['clean_children2'] = credit_scoring['clean_children'].astype('float')
children_pivot = pd.pivot_table(
    credit_scoring, index = 'clean_children2', columns = 'debt', values = 'clean_children', aggfunc = 'count', margins = 'True').reset_index()
children_pivot

debt,clean_children2,0,1,All
0,0.0,13086.0,1063.0,14149
1,1.0,4420.0,445.0,4865
2,2.0,1929.0,202.0,2131
3,3.0,303.0,27.0,330
4,4.0,37.0,4.0,41
5,5.0,9.0,NaN,9
6,All,19784.0,1741.0,21525


We got NaN for customers with 5 children. Let's check it in the data quickly. 

In [33]:
x = credit_scoring.loc[credit_scoring['debt'] == 1,]
x.loc[x['clean_children'] == 5,]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_clean,clean_income,grouped_income,grouped_purpose,clean_children,clean_children2


We see that there are no such values - great! The NaN value is not a mistake!

Now we will add a new column with the percentage of the dafult_rate because absolute numbers don't tell us anything.

In [34]:
children_pivot['default_rate'] = (children_pivot[1] / children_pivot['All']) *100
children_pivot


debt,clean_children2,0,1,All,default_rate
0,0.0,13086.0,1063.0,14149,7.512898
1,1.0,4420.0,445.0,4865,9.146968
2,2.0,1929.0,202.0,2131,9.479118
3,3.0,303.0,27.0,330,8.181818
4,4.0,37.0,4.0,41,9.756098
5,5.0,9.0,NaN,9,NaN
6,All,19784.0,1741.0,21525,8.088269


### Conclusion

Customers with no children are the safest in terms of repaying a loan. After that customers with 3 children, and surprisingly, customers with 1 child or 2 children are in the most untrusted customers among with costumers with 4 children. But generally, the more children a customer has, the bigger chance he will default a loan. . 

- Is there a relation between marital status and repaying a loan on time?

We will use pivot_table method.

In [35]:
family_status_pivot = pd.pivot_table(
    credit_scoring, index = 'family_status', columns = 'debt', values = 'family_status_id', aggfunc = 'count', margins = 'True').reset_index()
family_status_pivot

debt,family_status,0,1,All
0,civil partnership,3789,388,4177
1,divorced,1110,85,1195
2,married,11449,931,12380
3,unmarried,2539,274,2813
4,widow / widower,897,63,960
5,All,19784,1741,21525


Now we will add a new column with the percentage of the dafult_rate because absolute numbers don't tell us anything.

In [36]:
family_status_pivot['default_rate'] = (family_status_pivot[1] / family_status_pivot['All']) *100
family_status_pivot

debt,family_status,0,1,All,default_rate
0,civil partnership,3789,388,4177,9.288963
1,divorced,1110,85,1195,7.112971
2,married,11449,931,12380,7.520194
3,unmarried,2539,274,2813,9.740491
4,widow / widower,897,63,960,6.562500
5,All,19784,1741,21525,8.088269


### Conclusion

It seems that unmarried customers have the highest rate of loan default. It's also interesting to see that divorced and married people have the almost the same ranking. 
The unmarried category needs clarification because divorced people are also unmarried, so in the real-world we would ask the bank to clarify that. 
In general, it seems that married and divorced people default their loans in the same percentage, and that unmarried and the ones who are in a civil partnership are with the highest chance to default a loan. 
a widow or widower is the safest customer among the groups that we examined. 

- Is there a relation between income level and repaying a loan on time?

In [37]:
clean_income_pivot = pd.pivot_table(
    credit_scoring, index = 'grouped_income', columns = 'debt', values = 'clean_income', aggfunc = 'count', margins = 'True').reset_index()
clean_income_pivot

debt,grouped_income,0,1,All
0,middle class,5040,483,5523
1,millionaire,4995,386,5381
2,poor,4955,427,5382
3,rich,4794,445,5239
4,All,19784,1741,21525


Now we will add a new column with the percentage of the dafult_rate because absolute numbers don't tell us anything.

In [38]:
clean_income_pivot['default_rate'] = (clean_income_pivot[1] / clean_income_pivot['All']) *100
clean_income_pivot

debt,grouped_income,0,1,All,default_rate
0,middle class,5040,483,5523,8.745247
1,millionaire,4995,386,5381,7.173388
2,poor,4955,427,5382,7.933854
3,rich,4794,445,5239,8.493987
4,All,19784,1741,21525,8.088269


### Conclusion

Here we see that poor and millionaire people are the safest customers in terms of repaying a loan, while the middle class and the rich are the most untrusted. Nevertheless, the gap between the values is 1.57% at its top, so roughly speaking we don't have such a big difference between the income types. 

- How do different loan purposes affect on-time repayment of the loan?

In [39]:
purpose_pivot = pd.pivot_table(
    credit_scoring, index = 'grouped_purpose', columns = 'debt', values = 'purpose', aggfunc = 'count', margins = 'True').reset_index()
purpose_pivot

debt,grouped_purpose,0,1,All
0,car,3912,403,4315
1,education,3652,370,4022
2,housing,10058,782,10840
3,wedding,2162,186,2348
4,All,19784,1741,21525


Now we will add a new column with the percentage of the dafult_rate because absolute numbers don't tell us anything.

In [40]:
purpose_pivot['default_rate'] = (purpose_pivot[1] / purpose_pivot['All']) *100
purpose_pivot

debt,grouped_purpose,0,1,All,default_rate
0,car,3912,403,4315,9.339513
1,education,3652,370,4022,9.199403
2,housing,10058,782,10840,7.214022
3,wedding,2162,186,2348,7.921635
4,All,19784,1741,21525,8.088269


### Conclusion

Customers that take a loan for buying a house or to fund a wedding, are the most trustable, while customers that took a loan for a car or education are the riskiest.

## General conclusion

After answering all of the questions, we figured out that the best categories in terms of a loan replying are a number of children, family_status, and purpose. Customers with no children, a widow or a widower that buying a house are the best loaners for the bank (such a sad customer  😅 ).

Also, there is no difference between married and divorced in they ability to repay a loan - it's important data also from sociological point of view. Many people are afraid of divorce due to financial constraints on the part of the financial entities and continue to live together. So a bank that has such a figure, can give the same conditions to both groups, and on the way to being social and fair. This way, the bank's image will also rise among society. 


The worst customer for the bank is someone with 4 kids, unmarried who wants to buy a car. Education is also a category with high rick to default a loan. 

So we could highly reccomand to the bank to lower condition for a wedding puprpose loan from 2 reasons:
1. Loans for wedding purposes are at the lower group of risk for defaulting a loan. 
2. Married costomers are also at the lower group of risk for defaulting a loan.

**Meaning:** The loan for wedding purpose are most likely to be repyed, so it's beneficial from economical point of view. 

On the other hand, we would recommend putting costumers with kids, costumers that taking a loan for a car or education or customers that are in uninstitutionalized relationship (unmarried and civil partnership)  in a "risky folder" and monitor them closely. 
 